In [1]:
# Chargement des librairies
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('./fer2013.csv')

width, height = 48, 48

datapoints = data['pixels'].tolist()

# définir les données pour l'entraînement
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

# Définir des labels de l'entraînement
y = pd.get_dummies(data['emotion']).values

# Stockage des X et Y en utilisant numpy
np.save('fdataX', X)
np.save('flabels', y)

print("Preprocessing Done")
print("Number of Features: "+str(len(X[0])))
print("Number of Labels: "+ str(len(y[0])))
print("Number of examples in dataset:"+str(len(X)))
print("X,y stored in fdataX.npy and flabels.npy respectively")

Preprocessing Done
Number of Features: 48
Number of Labels: 7
Number of examples in dataset:35887
X,y stored in fdataX.npy and flabels.npy respectively


In [2]:
# Chargement des données
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

num_features = 64
num_labels = 7
batch_size = 64
epochs = 35
width, height = 48, 48

# Chargement des données
x = np.load('./fdataX.npy')
y = np.load('./flabels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

#for xx in range(10):
#    plt.figure(xx)
#    plt.imshow(x[xx].reshape((48, 48)), interpolation='none', cmap='gray')
#plt.show()

# Fractionnement des données d'entraînement , validation et de test


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

# Sauvegarde des données test
np.save('modXtest', X_test)
np.save('modytest', y_test)

In [3]:
# Conception du modèle
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

# Utilisation de l'algorithme SOFTMAX pour la  classification multiple

In [4]:
# Complilation du modèle avec un optimiseur et une catégorisation des pertes
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

# Sauvegarde du modèle
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")
print("Saved model to disk")


Epoch 1/35
455/455 [==============================] - 824s 2s/step - loss: 1.9880 - accuracy: 0.2152 - val_loss: 1.8283 - val_accuracy: 0.2594
Epoch 2/35
455/455 [==============================] - 925s 2s/step - loss: 1.8240 - accuracy: 0.2520 - val_loss: 1.7527 - val_accuracy: 0.2839
Epoch 3/35
455/455 [==============================] - 1123s 2s/step - loss: 1.7268 - accuracy: 0.3043 - val_loss: 1.6046 - val_accuracy: 0.3588
Epoch 4/35
455/455 [==============================] - 1160s 3s/step - loss: 1.6118 - accuracy: 0.3618 - val_loss: 1.4516 - val_accuracy: 0.4288
Epoch 5/35
455/455 [==============================] - 1142s 3s/step - loss: 1.5102 - accuracy: 0.4088 - val_loss: 1.4395 - val_accuracy: 0.4149
Epoch 6/35
455/455 [==============================] - 1147s 3s/step - loss: 1.4515 - accuracy: 0.4288 - val_loss: 1.3445 - val_accuracy: 0.4762
Epoch 7/35
455/455 [==============================] - 1142s 3s/step - loss: 1.3993 - accuracy: 0.4568 - val_loss: 1.3390 - val_accuracy: 0

In [5]:
# Chargement des librairies
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# Chargement du modèle
model = model_from_json(open("fer.json", "r").read())
model.load_weights('fer.h5')

# Chargement de la cascade 
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    # Capture de l'image et retour sous forme booléenne
    ret,test_img=cap.read() 
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        # Zone du visage
        roi_gray=gray_img[y:y+w,x:x+h]# Zone du visage
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        # Trouve un tableau indexé
        max_index = np.argmax(predictions[0])

        # Labels mis dans un tuple
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)


    # Appuyer sur q pour quitter 
    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>